In [4]:
import pyodbc
print(pyodbc.drivers())

['SQL Server']


In [5]:
# load environment variables
import os
from dotenv import load_dotenv
load_dotenv()
import pyodbc

PWD = os.getenv('PWD')
MSQL_Server = os.getenv('MSQL_Server')
USERNAME = os.getenv('USERNAME')

conn = pyodbc.connect(
                'DRIVER={SQL Server};'
                'SERVER={MSQL_Server};'
                'DATABASE=SAS;'
                'UID={USERNAME};'
                'PWD={PWD};'
            )
cursor = conn.cursor()

OperationalError: ('08001', '[08001] [Microsoft][ODBC SQL Server Driver][DBNETLIB]SQL Server existiert nicht oder Zugriff verweigert. (17) (SQLDriverConnect); [08001] [Microsoft][ODBC SQL Server Driver][DBNETLIB]ConnectionOpen (Connect()). (53)')